<a href="https://colab.research.google.com/github/sidsanc/258_DeepLearning/blob/main/CatchUp_Assignment/Part2_KerasNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part 2: Keras NLP**

## a. Basic keras nlp tasks

In [2]:
!pip install keras_nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 44.6 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import keras_nlp
import tensorflow_datasets as tfds

### Inference with a pretrained classifier

In [4]:
snli = tfds.load('snli')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/snli/1.1.0.incompleteW4R3P7/snli-validation.tfrecord*...:   0%|          |…

Generating train examples...:   0%|          | 0/550152 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/snli/1.1.0.incompleteW4R3P7/snli-train.tfrecord*...:   0%|          | 0/55…

Dataset snli downloaded and prepared to /root/tensorflow_datasets/snli/1.1.0. Subsequent calls will reuse this data.


In [5]:
snli

{Split('test'): <_PrefetchDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'premise': TensorSpec(shape=(), dtype=tf.string, name=None)}>,
 Split('validation'): <_PrefetchDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'premise': TensorSpec(shape=(), dtype=tf.string, name=None)}>,
 Split('train'): <_PrefetchDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'premise': TensorSpec(shape=(), dtype=tf.string, name=None)}>}

In [6]:
tfds.as_dataframe(snli['train'].take(20))

,hypothesis,label,premise
0,b'Washing clothes on a camping trip.',1,b'A man washes or dies clothes in a primitive setting.'
1,b'A woman walking alone.',2,b'A woman is walking her baby with a stroller at the local park.'
2,b'The woman is thinking.',-1,b'This is a pensive women in a grassy setting.'
3,b'The woman is practicing to enter a roller derby.',1,b'Woman in red shirt and white cap rollerblading on gray surface.'
4,b'The kids are singing in a school play.',1,b'A group of African children dressed in mismatched clothing sing and play instruments.'
5,b'There are motorcycles in a lot.',0,b'A group of motorcycles and scooters are parked together in a lot in the middle of a street.'
6,b'A man performs a circus act.',1,b'A man twirling a flaming baton.'
7,b'The woman was crafting a wooden toy for her child.',1,b'A woman with a gray shirt and dirty white apron works with wood.'
8,b'There were many law enforcement around.',1,b'Law enforcement that is riding in a large tactical for truck in an urban area.'
9,"b""A creepy guy is putting his favorite photos of the woman he's stalking up in his shrine to her.""",2,b'A person is hanging up pictures of women with a few onlookers watching surrounded by bikes.'


In [7]:
snli_train = tfds.load('snli', split='train[:20%]')
snli_val = tfds.load('snli', split='validation')
snli_test = tfds.load('snli', split='test')

In [8]:
snli_test

<_PrefetchDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'premise': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [9]:
def to_dataframe(dataset):
    dataset = tfds.as_dataframe(dataset)
    dataset['hypothesis'] = dataset['hypothesis'].str.decode("utf-8")
    dataset['premise'] = dataset['premise'].str.decode("utf-8")
    return dataset

def preprocess_dataset(dataset, preprocessor): 
    dataset = to_dataframe(dataset)
    dataset.drop(dataset[dataset['label'] == -1].index, inplace=True)
    x = preprocessor((dataset['hypothesis'], dataset['premise']))
    y = np.array(dataset.label)
    return x, y

In [10]:
bert_preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_tiny_en_uncased")

x_train, y_train = preprocess_dataset(snli_train, bert_preprocessor)
x_test, y_test = preprocess_dataset(snli_test, bert_preprocessor)
x_val, y_val = preprocess_dataset(snli_val, bert_preprocessor)

231508/231508 [==============================] - 0s 0us/step


In [ ]:
bert_clf = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", 
    num_classes=3, 
    preprocessor=None)

bert_clf.summary()

In [ ]:
bert_clf.fit(x = x_train, y = y_train, validation_data = (x_val, y_val), epochs=1, batch_size=512)

### Fine tuning a pretrained BERT backbone

In [ ]:
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=2,
)
classifier.fit(
    imdb_train,
    validation_data=imdb_test,
    epochs=1,
)

### Fine tuning with user-controlled preprocessing

In [23]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!# Remove unsupervised examples
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  9263k      0  0:00:08  0:00:08 --:--:-- 15.3M


In [24]:
import tensorflow as tf
from tensorflow import keras
import keras_nlp

BATCH_SIZE = 16
imdb_train = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=BATCH_SIZE,
)
imdb_test = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=BATCH_SIZE,
)

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_tiny_en_uncased",
    sequence_length=512,
)

imdb_train_cached = (
    imdb_train.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)
imdb_test_cached = (
    imdb_test.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    preprocessor=None,
)
classifier.fit(
    imdb_train_cached,
    validation_data=imdb_test_cached,
    epochs=3,
)

### Fine Tuning with a custom model

In [25]:
print(imdb_train.unbatch().take(1).get_single_element())

(<tf.Tensor: shape=(), dtype=string, numpy=b"A really cool flick. A must for any music snob. You don't really have to know about the bands to enjoy the movie. Before the movie, I only heard only two songs from the Dandy Warhols. The only thing is required is an open mind. <br /><br />The movie centers around the Brian Jonestown Massacre. The Dandy Warhols have a role in the film, as the 'rival band,' but they are second fiddle to the BJM. The Dandy Warhols don't play as big of a role in the film as I originally guessed, but then again, they didn't have the element of excitement and unpredictability of the BJM.You can't help but be fascinated by the band and its very charismatic front man, Anton Newcombe. By itself, it's an insightful film and study on the music industry. Just watch this film and enjoy.">, <tf.Tensor: shape=(), dtype=int32, numpy=1>)


In [ ]:
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=2,
)
classifier.fit(
    imdb_train,
    validation_data=imdb_test,
    epochs=1,
)

In [27]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_tiny_en_uncased")
backbone = keras_nlp.models.BertBackbone.from_preset("bert_tiny_en_uncased")

In [29]:
imdb_train_preprocessed = (
    imdb_train.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)
imdb_test_preprocessed = (
    imdb_test.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

In [28]:
backbone.trainable = False

In [30]:
inputs = backbone.input
sequence = backbone(inputs)["sequence_output"]
for _ in range(2):
    sequence = keras_nlp.layers.TransformerEncoder(
        num_heads=2,
        intermediate_dim=512,
        dropout=0.1,
    )(sequence)

outputs = keras.layers.Dense(3)(sequence[:, backbone.cls_token_index, :])

In [ ]:
model = keras.Model(inputs, outputs)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.AdamW(5e-5),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

model.summary()

In [32]:
model.fit(
    imdb_train_preprocessed,
    validation_data=imdb_test_preprocessed,
    epochs=3,
)

Epoch 1/3
1563/1563 [==============================] - 150s 90ms/step - loss: 0.5821 - sparse_categorical_accuracy: 0.6957 - val_loss: 0.5525 - val_sparse_categorical_accuracy: 0.7223
Epoch 2/3
1563/1563 [==============================] - 110s 70ms/step - loss: 0.4774 - sparse_categorical_accuracy: 0.7724 - val_loss: 0.4623 - val_sparse_categorical_accuracy: 0.7807
Epoch 3/3
1563/1563 [==============================] - 110s 70ms/step - loss: 0.4360 - sparse_categorical_accuracy: 0.7972 - val_loss: 0.4308 - val_sparse_categorical_accuracy: 0.8036


### Pretraining a backbone model

In [ ]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_tiny_en_uncased",
    sequence_length=256,
)
packer = preprocessor.packer
tokenizer = preprocessor.tokenizer

# keras.Layer to replace some input tokens with the "[MASK]" token
masker = keras_nlp.layers.MaskedLMMaskGenerator(
    vocabulary_size=tokenizer.vocabulary_size(),
    mask_selection_rate=0.25,
    mask_selection_length=64,
    mask_token_id=tokenizer.token_to_id("[MASK]"),
    unselectable_token_ids=[
        tokenizer.token_to_id(x) for x in ["[CLS]", "[PAD]", "[SEP]"]
    ],
)


def preprocess(inputs, label):
    inputs = preprocessor(inputs)
    masked_inputs = masker(inputs["token_ids"])
  
    features = {
        "token_ids": masked_inputs["token_ids"],
        "segment_ids": inputs["segment_ids"],
        "padding_mask": inputs["padding_mask"],
        "mask_positions": masked_inputs["mask_positions"],
    }
    labels = masked_inputs["mask_ids"]
    weights = masked_inputs["mask_weights"]
    return features, labels, weights


pretrain_ds = imdb_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)
pretrain_val_ds = imdb_test.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

# Tokens with ID 103 are "masked"
print(pretrain_ds.unbatch().take(1).get_single_element())

In [ ]:
backbone = keras_nlp.models.BertBackbone(
    vocabulary_size=tokenizer.vocabulary_size(),
    num_layers=2,
    num_heads=2,
    hidden_dim=128,
    intermediate_dim=512,
)

mlm_head = keras_nlp.layers.MaskedLMHead(
    embedding_weights=backbone.token_embedding.embeddings,
)

inputs = {
    "token_ids": keras.Input(shape=(None,), dtype=tf.int32),
    "segment_ids": keras.Input(shape=(None,), dtype=tf.int32),
    "padding_mask": keras.Input(shape=(None,), dtype=tf.int32),
    "mask_positions": keras.Input(shape=(None,), dtype=tf.int32),
}

# Encoded token sequence
sequence = backbone(inputs)["sequence_output"]

outputs = mlm_head(sequence, mask_positions=inputs["mask_positions"])


pretraining_model = keras.Model(inputs, outputs)
pretraining_model.summary()
pretraining_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.experimental.AdamW(learning_rate=5e-4),
    weighted_metrics=keras.metrics.SparseCategoricalAccuracy(),
    jit_compile=True,
)


pretraining_model.fit(
    pretrain_ds,
    validation_data=pretrain_val_ds,
    epochs=6,  
)

### Build and train your own transformer from scratch

In [ ]:
vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    imdb_train.map(lambda x, y: x),
    vocabulary_size=20_000,
    lowercase=True,
    strip_accents=True,
    reserved_tokens=["[PAD]", "[START]", "[END]", "[MASK]", "[UNK]"],
)
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    lowercase=True,
    strip_accents=True,
    oov_token="[UNK]",
)

In [ ]:
packer = keras_nlp.layers.StartEndPacker(
    start_value=tokenizer.token_to_id("[START]"),
    end_value=tokenizer.token_to_id("[END]"),
    pad_value=tokenizer.token_to_id("[PAD]"),
    sequence_length=512,
)


def preprocess(x, y):
    token_ids = packer(tokenizer(x))
    return token_ids, y


imdb_preproc_train_ds = imdb_train.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)
imdb_preproc_val_ds = imdb_test.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

print(imdb_preproc_train_ds.unbatch().take(1).get_single_element())

In [ ]:
token_id_input = keras.Input(
    shape=(None,),
    dtype="int32",
    name="token_ids",
)
outputs = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=len(vocab),
    sequence_length=packer.sequence_length,
    embedding_dim=64,
)(token_id_input)
outputs = keras_nlp.layers.TransformerEncoder(
    num_heads=2,
    intermediate_dim=128,
    dropout=0.1,
)(outputs)
# Use "[START]" token to classify
outputs = keras.layers.Dense(2)(outputs[:, 0, :])
model = keras.Model(
    inputs=token_id_input,
    outputs=outputs,
)

model.summary()

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.experimental.AdamW(5e-5),
    metrics=keras.metrics.SparseCategoricalAccuracy(),
    jit_compile=True,
)
model.fit(
    imdb_preproc_train_ds,
    validation_data=imdb_preproc_val_ds,
    epochs=6,
)